In [ ]:
import copy
import pandas as pd

In [ ]:
raw_data = pd.read_excel('23fall.xlsx')
raw_data.head(3)

In [ ]:
stat_data = raw_data.iloc[1:].loc[:, ['15. 总GPA', 'Unnamed: 16', '18. 你的语言考试是', '19. 你的语言考试成绩为', '20. 你的GRE成绩是',\
                                      'Unnamed: 23', 'Unnamed: 24', '22. 你获得了哪些项目的 offer', '23. 你收到了哪些项目的拒信']]
stat_data.head(2)

In [ ]:
class Stat(object):
    def __init__(self):
        self.stat = {}
        self.program_stat = {'num': 0, 'gpa4': [0, 0, 4], 'gpa100': [0, 0, 100], 'toefl_overall': [0, 0, 120], 'toefl_speaking': [0, 0, 30],\
                             'ielts_overall': [0, 0, 9], 'ielts_speaking': [0, 0, 9], 'gre': [0, 0, 340], 'aw': [0, 0, 6]}
    
    def add_stat(self, university, program, status, gpa4, gpa100, language, overall, speaking, gre, aw):
        record = {'gpa4': gpa4, 'gpa100': gpa100, 'toefl_overall': None, 'toefl_speaking': None,\
                  'ielts_overall': None, 'ielts_speaking': None, 'gre': gre, 'aw': aw}
        if language == 0:
            record['toefl_overall'] = overall
            record['toefl_speaking'] = speaking
        elif language == 1:
            record['ielts_overall'] = overall
            record['ielts_speaking'] = speaking
        if university not in self.stat.keys():
            self.stat[university] = {program: {'ad': copy.deepcopy(self.program_stat), 'rej': copy.deepcopy(self.program_stat)}}
        elif program not in self.stat[university].keys():
            self.stat[university][program] = {'ad': copy.deepcopy(self.program_stat), 'rej': copy.deepcopy(self.program_stat)}
        for key in self.stat[university][program][status].keys():
            if key == 'num' or not record[key]:
                continue
            self.stat[university][program][status][key][0] += 1
            self.stat[university][program][status][key][1] += record[key]
            self.stat[university][program][status][key][2] = min(self.stat[university][program][status][key][2], record[key])
        self.stat[university][program][status]['num'] += 1
    
    def conclude_stat(self):
        for university in self.stat.keys():
            for program in self.stat[university].keys():
                self.stat[university][program]['total'] = self.stat[university][program]['ad']['num'] + self.stat[university][program]['rej']['num']
                self.stat[university][program]['ad_num'] = self.stat[university][program]['ad']['num']
                self.stat[university][program]['ad_rate'] = self.stat[university][program]['ad']['num'] / self.stat[university][program]['total']
                if self.stat[university][program]['ad']['num'] == 0:
                    self.stat[university][program]['ad'] = self.stat[university][program]['rej']
                for key in self.stat[university][program]['ad'].keys():
                    if key == 'num':
                        continue
                    if self.stat[university][program]['ad'][key][0]:
                        self.stat[university][program]['ad'][key][1] /= self.stat[university][program]['ad'][key][0]
                    else:
                        self.stat[university][program]['ad'][key][1] = None
                        self.stat[university][program]['ad'][key][2] = None
    
    def print_stat(self, sheet_name):
        sheet = []
        for university in self.stat.keys():
            for program in self.stat[university].keys():
                sheet.append([university, program, self.stat[university][program]['ad_num'], self.stat[university][program]['total'],\
                            self.stat[university][program]['ad_rate'], self.stat[university][program]['ad']['gpa4'][1],\
                            self.stat[university][program]['ad']['gpa100'][1], self.stat[university][program]['ad']['gre'][1],\
                            self.stat[university][program]['ad']['aw'][1], self.stat[university][program]['ad']['toefl_overall'][1],\
                            self.stat[university][program]['ad']['toefl_speaking'][1], self.stat[university][program]['ad']['ielts_overall'][1],\
                            self.stat[university][program]['ad']['ielts_speaking'][1], self.stat[university][program]['ad']['gpa4'][2],\
                            self.stat[university][program]['ad']['gpa100'][2], self.stat[university][program]['ad']['gre'][2],\
                            self.stat[university][program]['ad']['aw'][2], self.stat[university][program]['ad']['toefl_overall'][2],\
                            self.stat[university][program]['ad']['toefl_speaking'][2], self.stat[university][program]['ad']['ielts_overall'][2],\
                            self.stat[university][program]['ad']['ielts_speaking'][2]])
        sheet = pd.DataFrame(sheet, columns=['学校', '项目', '录取人数', '申请人数', '录取率', '平均GPA', '平均百分', '平均GRE',\
                                             '平均AW', '平均托福', '平均口语(T)', '平均雅思', '平均口语(I)', '最低GPA', '最低百分',\
                                             '最低GRE', '最低AW', '最低托福', '最低口语(T)', '最低雅思', '最低口语(I)'])
        with pd.ExcelWriter(sheet_name) as writer:
            sheet.to_excel(writer)

In [ ]:
def process_record(record):
    if '#' in record:
        university, program = tuple(record.split('#'))
    elif '@' in record:
        program, university = tuple(record.split('@'))
    else:
        print('Invalid format', record)
        university, program = None, None
    return university, program

In [ ]:
stat = Stat()

for idx, data in stat_data.iterrows():
    gpa4 = float(data['15. 总GPA'].replace('～', '').replace('+', ''))
    gpa100 = float(data['Unnamed: 16'].replace('～', '').replace('+', ''))
    # print(gpa4, gpa100)
    language = 0 if data['18. 你的语言考试是'] == 'TOEFL' else 1 if data['18. 你的语言考试是'] == 'IELTS' else 2
    if language == 2:
        overall = None
        speaking = None
    else:
        speaking = -1
        data['19. 你的语言考试成绩为'] = data['19. 你的语言考试成绩为'].replace('(', ' ').replace(')', '').replace('～', '')
        try:
            overall = float(data['19. 你的语言考试成绩为'].split()[0])
        except:
            if len(data['19. 你的语言考试成绩为'].split()) == 4:
                overall = float(data['19. 你的语言考试成绩为'].split()[0][-2:]) + float(data['19. 你的语言考试成绩为'].split()[1][-2:])\
                        + float(data['19. 你的语言考试成绩为'].split()[2][-2:]) + float(data['19. 你的语言考试成绩为'].split()[3][-2:])
                speaking = float(data['19. 你的语言考试成绩为'].split()[2][-2:])
            elif float(data['19. 你的语言考试成绩为'][:3]) <= 120:
                overall = float(data['19. 你的语言考试成绩为'][:3])
                speaking = None
            else:
                print('Invalid format', data['19. 你的语言考试成绩为'])
                continue
        if len(data['19. 你的语言考试成绩为'].split()) == 1:
            speaking = None
        elif speaking == None:
            pass
        else:
            try:
                speaking = float(data['19. 你的语言考试成绩为'].split()[3][-2:]) if language == 0 else float(data['19. 你的语言考试成绩为'].split()[3][-3:])
            except:
                print(overall, 'Invalid format', data['19. 你的语言考试成绩为'])
                continue
    # print(overall, speaking)
    if pd.isna(data['20. 你的GRE成绩是']):
        gre = None
        aw = None
    else:
        gre = int(data['20. 你的GRE成绩是']) + int(data['Unnamed: 23'])
        aw = float(data['Unnamed: 24'])
    # print(gre, aw)
    offer_list = data['22. 你获得了哪些项目的 offer'].split('\n')
    rej_list = data['23. 你收到了哪些项目的拒信'].split('\n')
    for offer in offer_list:
        university, program = process_record(offer)
        if university == None and program == None:
            continue
        print(university, program)
        stat.add_stat(university, program, 'ad', gpa4, gpa100, language, overall, speaking, gre, aw)
    for rej in rej_list:
        university, program = process_record(rej)
        if university == None and program == None:
            continue
        print(university, program)
        stat.add_stat(university, program, 'rej', gpa4, gpa100, language, overall, speaking, gre, aw)
    print('')

In [ ]:
stat.conclude_stat()
stat.print_stat('stat.xlsx')